In [74]:
import numpy as np

def shift_latt(latt):
    
    grid = latt
        
    # center (0,0)
    # stays constant
        
    # east (0,1)
    ch = 1
    grid[:, :, ch] = np.roll(grid[:, :, ch], shift=1, axis=1)
        
    # north (-1, 0)
    ch = 2
    grid[:, :, ch] = np.roll(grid[:, :, ch], shift=-1, axis=0)
        
    # west (0, -1)
    ch = 3
    grid[:, :, ch] = np.roll(grid[:, :, ch], shift=-1, axis=1)
        
    # south (1, 0)
    ch = 4
    grid[:, :, ch] = np.roll(grid[:, :, ch], shift=1, axis=0)
        
    # north-east(-1, 1)
    ch = 5
    grid[:, :, ch] = np.roll(np.roll(grid[:, :, ch], shift=-1, axis=0), shift=1, axis=1)
        
    # north-west(-1, -1)
    ch = 6
    grid[:, :, ch] = np.roll(np.roll(grid[:, :, ch], shift=-1, axis=0), shift=-1, axis=1)
        
    # south-west(1, -1)
    ch = 7
    grid[:, :, ch] = np.roll(np.roll(grid[:, :, ch], shift=1, axis=0), shift=-1, axis=1)
        
    # south-east(1, 1)
    ch = 8
    grid[:, :, ch] = np.roll(np.roll(grid[:, :, ch], shift=1, axis=0), shift=1, axis=1)
            
    return grid


'''
def optimierung(mat):
    
    matShift = mat
    
    c = np.array([[0, 0, -1, 0, 1, 1, -1, -1, 1], [0, -1, 0, 1, 0, -1, -1, 1, 1]])
    #c1 = np.array([[0, 1, 0, -1, 0, 1, -1, -1, 1], [0, 0, 1, 0, -1, 1, 1, -1, -1]])
    
    # Funktionieren nicht mit der Schleife, da i sonst auserhalb des Arrays zugreift
    #c2 = np.array([[0,0], [1,0], [0,1], [-1,0], [0,-1], [1,1], [-1,1], [-1,-1], [1,-1]])
    #c3 = np.array([[0,0], [0,1], [-1,0], [0,-1], [1,0], [-1,1], [-1,-1], [1,-1], [1,1]])

    # von 1 bis 8 für die Channels
    for i in range (1,9):
        print (i)
        print (c[1,i])
        print (c[0,i])
        matShift = np.roll(np.roll(matShift[:, :, i], shift=c[1,i], axis=0), shift=c[0,i], axis=1)
    return matShift
'''

def sum_occ(mat):
    occ = np.sum(mat, axis=2, dtype=float)
    return occ



def calc_j(c, mat):
    j = np.dot(mat, c)
    
    # Alternative
    #j1 = np.einsum('abc, cd-> abd', mat, c)
    
    return j


def calc_avg_vel(rho, j):
    u = j / rho.reshape(rows, cols, 1)
    
    # Alternative
    #u1 = (j.T / rho.T).T
    
    return u

def calc_equilibrium(w, rho, u, c1):
    
    
    ch = len(w)
    rows = rho.shape[0]
    cols = rho.shape[1]

    f_eq = np.zeros((rows,cols,ch))
    
    
    for k in range(rows):
        for j in range(cols):
            for i in range(ch):
                node = (k,j)
                
                rho_temp = rho[node]
                u_temp = u[node].reshape((-1,1)).T
                c_temp = c1[i].reshape((-1,1))
                
                # Optimierung die 3 For.Schleifen entfernen
                f_eq[k,j,i] = w[i] * rho_temp * (1 + 3 * u_temp.dot(c_temp) + 9/2 * u_temp.dot(c_temp)**2 - 3/2 * u_temp.dot(u_temp.T))
    
    return f_eq


rows = 3
cols = 3
q = 9
timesteps = 500

# c = np.array([[0, 0, -1, 0, 1, -1, -1, 1, 1], [0, 1, 0, -1, 0, 1, -1, -1, 1]])

c1 = np.array([[0,0], [0,1], [-1,0], [0,-1], [1,0], [-1,1], [-1,-1], [1,-1], [1,1]])

latt = np.zeros((rows, cols, q), dtype = float)

# initial lattice occupation
latt[:,:,0] = 4 / 9
latt[:,:,1] = 1 / 9
latt[:,:,2] = 1 / 9
latt[:,:,3] = 1 / 9
latt[:,:,4] = 1 / 9
latt[:,:,5] = 1 / 36
latt[:,:,6] = 1 / 36
latt[:,:,7] = 1 / 36
latt[:,:,8] = 1 / 36

ch = 3

w = np.array([4/9, 1/9, 1/9, 1/9, 1/9, 1/36, 1/36, 1/36, 1/36])


omega = 0.1
u = np.zeros((rows, cols, 2))
u[0,:,0] = 1.1
u[1,:,0] = 0
u[2,:,0] = -1.1

u[:,:,1] = 0.02

# shear wave decay init
rho = np.full((rows,cols), 0.1)

epsilon = 0.01


val = (2 * np.pi * rows) / (u[:,:,0] - 1)

print(val.shape)

print(np.sin(val))

#print(0.1 + epsilon * val[1])

print(rho)
rho += epsilon * val


print(rho)

#rho = sum_occ(latt)
#print(rho.shape)
#print(rho, '\n', '\n')

#j = calc_j(c1, latt)
#print(j[0,0], j[1,1])

#u = calc_avg_vel(rho, j)
#print(u)

#f_eq = calc_equilibrium(rho, u, c1)
#print(f_eq)

for t in range(timesteps):
    print(latt[:,:,ch], '\n')
    latt = shift_latt(latt)
    
    j = calc_j(c1, latt)
    f_eq = calc_equilibrium(w, rho, u, c1)
    u = calc_avg_vel(rho, j)
    rho = sum_occ(latt)
    
    
    #print(f_eq)
    
    latt += omega * (f_eq - latt)
    

(3, 3)
[[ -1.63667283e-13  -1.63667283e-13  -1.63667283e-13]
 [  7.34788079e-16   7.34788079e-16   7.34788079e-16]
 [ -4.33883739e-01  -4.33883739e-01  -4.33883739e-01]]
[[ 0.1  0.1  0.1]
 [ 0.1  0.1  0.1]
 [ 0.1  0.1  0.1]]
[[ 1.98495559  1.98495559  1.98495559]
 [-0.08849556 -0.08849556 -0.08849556]
 [ 0.01024021  0.01024021  0.01024021]]
[[ 0.11111111  0.11111111  0.11111111]
 [ 0.11111111  0.11111111  0.11111111]
 [ 0.11111111  0.11111111  0.11111111]] 

[[ 0.08072829  0.08072829  0.08072829]
 [ 0.09907453  0.09907453  0.09907453]
 [ 0.09990058  0.09990058  0.09990058]] 

[[ 0.08376657  0.08376657  0.08376657]
 [ 0.10027819  0.10027819  0.10027819]
 [ 0.10102163  0.10102163  0.10102163]] 

[[ 0.08410907  0.08410907  0.08410907]
 [ 0.10169486  0.10169486  0.10169486]
 [ 0.10140167  0.10140167  0.10140167]] 

[[ 0.08464853  0.08464853  0.08464853]
 [ 0.1020419   0.1020419   0.1020419 ]
 [ 0.10266119  0.10266119  0.10266119]] 

[[ 0.0868694   0.0868694   0.0868694 ]
 [ 0.10197179  0.1

 [ 0.10609651  0.10609651  0.10609651]] 

[[ 0.10595575  0.10595575  0.10595575]
 [ 0.1060948   0.1060948   0.1060948 ]
 [ 0.10609498  0.10609498  0.10609498]] 

[[ 0.1059587   0.1059587   0.1059587 ]
 [ 0.10609334  0.10609334  0.10609334]
 [ 0.10609349  0.10609349  0.10609349]] 

[[ 0.10596155  0.10596155  0.10596155]
 [ 0.10609193  0.10609193  0.10609193]
 [ 0.10609205  0.10609205  0.10609205]] 

[[ 0.10596432  0.10596432  0.10596432]
 [ 0.10609056  0.10609056  0.10609056]
 [ 0.10609066  0.10609066  0.10609066]] 

[[ 0.10596699  0.10596699  0.10596699]
 [ 0.10608923  0.10608923  0.10608923]
 [ 0.10608931  0.10608931  0.10608931]] 

[[ 0.10596958  0.10596958  0.10596958]
 [ 0.10608795  0.10608795  0.10608795]
 [ 0.106088    0.106088    0.106088  ]] 

[[ 0.10597209  0.10597209  0.10597209]
 [ 0.10608671  0.10608671  0.10608671]
 [ 0.10608674  0.10608674  0.10608674]] 

[[ 0.10597452  0.10597452  0.10597452]
 [ 0.1060855   0.1060855   0.1060855 ]
 [ 0.10608552  0.10608552  0.10608552]] 

[[ 0.10604753  0.10604753  0.10604753]
 [ 0.10604902  0.10604902  0.10604902]
 [ 0.10604899  0.10604899  0.10604899]] 

[[ 0.10604757  0.10604757  0.10604757]
 [ 0.106049    0.106049    0.106049  ]
 [ 0.10604897  0.10604897  0.10604897]] 

[[ 0.1060476   0.1060476   0.1060476 ]
 [ 0.10604899  0.10604899  0.10604899]
 [ 0.10604896  0.10604896  0.10604896]] 

[[ 0.10604762  0.10604762  0.10604762]
 [ 0.10604897  0.10604897  0.10604897]
 [ 0.10604894  0.10604894  0.10604894]] 

[[ 0.10604765  0.10604765  0.10604765]
 [ 0.10604896  0.10604896  0.10604896]
 [ 0.10604893  0.10604893  0.10604893]] 

[[ 0.10604768  0.10604768  0.10604768]
 [ 0.10604894  0.10604894  0.10604894]
 [ 0.10604891  0.10604891  0.10604891]] 

[[ 0.10604771  0.10604771  0.10604771]
 [ 0.10604893  0.10604893  0.10604893]
 [ 0.1060489   0.1060489   0.1060489 ]] 

[[ 0.10604773  0.10604773  0.10604773]
 [ 0.10604892  0.10604892  0.10604892]
 [ 0.10604889  0.10604889  0.10604889]] 

[[ 0.10604776  0.10604776  0.10604776]
 


[[ 0.10604849  0.10604849  0.10604849]
 [ 0.10604852  0.10604852  0.10604852]
 [ 0.10604852  0.10604852  0.10604852]] 

[[ 0.10604849  0.10604849  0.10604849]
 [ 0.10604852  0.10604852  0.10604852]
 [ 0.10604852  0.10604852  0.10604852]] 

[[ 0.10604849  0.10604849  0.10604849]
 [ 0.10604852  0.10604852  0.10604852]
 [ 0.10604852  0.10604852  0.10604852]] 

[[ 0.10604849  0.10604849  0.10604849]
 [ 0.10604852  0.10604852  0.10604852]
 [ 0.10604852  0.10604852  0.10604852]] 

[[ 0.10604849  0.10604849  0.10604849]
 [ 0.10604852  0.10604852  0.10604852]
 [ 0.10604852  0.10604852  0.10604852]] 

[[ 0.10604849  0.10604849  0.10604849]
 [ 0.10604852  0.10604852  0.10604852]
 [ 0.10604852  0.10604852  0.10604852]] 

[[ 0.10604849  0.10604849  0.10604849]
 [ 0.10604852  0.10604852  0.10604852]
 [ 0.10604852  0.10604852  0.10604852]] 

[[ 0.10604849  0.10604849  0.10604849]
 [ 0.10604852  0.10604852  0.10604852]
 [ 0.10604852  0.10604852  0.10604852]] 

[[ 0.1060485   0.1060485   0.1060485 ]
